# Mapping Cadastral Data with GeoPandas
### Author: Jorge Monge
### Date: 2021-May-11

### Python script which uses GeoPandas to map cadastral data (http://www.catastro.minhap.es/) for province capitals in Spain

### Please note the Terms Of Use of the data at http://www.catastro.minhap.es/webinspire/documentos/Licencia.pdf

In [ ]:
import geopandas
import matplotlib.pyplot as plt
import contextily as ctx
import requests
from zipfile import ZipFile
import os
import feedparser
import unicodedata

#### Defining constants

In [ ]:
PROVINCE = "Cádiz"
PROV_CAPITAL = "Cádiz"
BUFFER_DIST = 10000
CADASTRAL_DATA_FEED = "http://www.catastro.minhap.es/INSPIRE/buildings/ES.SDGC.bu.atom.xml"

#### Normalize string values (remove diacritic accent marks)

In [ ]:
def removeAccentMarks(string):
    return ''.join(c for c in unicodedata.normalize('NFD', string)
                  if unicodedata.category(c) != 'Mn')

PROVINCE = removeAccentMarks(PROVINCE).lower()
PROV_CAPITAL = removeAccentMarks(PROVINCE).lower()

#### Parsing the ATOM feed
See http://www.catastro.minhap.es/webinspire/index.html

In [ ]:
feed = feedparser.parse(CADASTRAL_DATA_FEED)

#### Print provinces and link

In [ ]:
entries = feed.entries
discard = [(lambda e: print(e.title, e.link))(e) for e in feed.entries]

#### Parsing the provincial ATOM feed

Checking the parsing method output (assumming the provincial capitals' postal codes all end with '900'

In [ ]:

selected_province_feed = \
    [(lambda e: feedparser.parse(e.link)) (e) for e in entries if PROVINCE in removeAccentMarks(e.title).lower()][0]["entries"]

capital_feed = \
    [(lambda e: {"title": e["title"], "zipped_gml": e["link"]})
                 (e) for e in selected_province_feed if f"900-{PROV_CAPITAL.upper()} " in e["title"].upper()][0]
capital_feed

#### Creating a local directory to save the zipped GML file

In [ ]:
target_data_dir = os.path.join(os.getcwd(),"Downloaded_Data", f"{PROV_CAPITAL.lower()}")
if not os.path.exists(target_data_dir):
    os.makedirs(target_data_dir)
zipped_data = os.path.join(target_data_dir, f"{PROV_CAPITAL.lower()}.zip")

#### Downloading the zipped GML file

In [ ]:
url = capital_feed["zipped_gml"]
r = requests.get(url)

with open (zipped_data, 'wb') as f:
    f.write(r.content)

#### Creating a GeoDataFrame from the appropriate .gml file

In [ ]:
gml_file = [f for f in ZipFile(zipped_data).namelist() if "building.gml" in f][0]
gdf = geopandas.read_file(f"{zipped_data}!/{gml_file}")

#### Getting the center of the city from the geopandas repository

In [ ]:
# CS of the geocoding response = EPSG:4326
center_city_geocoding_response = geopandas.tools.geocode(PROV_CAPITAL)["geometry"]
# Reproject to Web Mercator
center_city_web_mercator = center_city_geocoding_response.to_crs("EPSG:3857")
center_city_web_mercator

#### Creating the buffer around the city center

In [ ]:
# Buffer around the center of the city
city_buffer = geopandas.GeoDataFrame({'geometry': center_city_web_mercator.buffer(BUFFER_DIST, resolution=16)})

In [ ]:
#### Plotting the intersection of the city buildings with the buffer, with a basemap.

In [ ]:
intersection = geopandas.overlay(gdf.to_crs(epsg=3857), city_buffer, how='intersection')
ax = intersection.plot(column="beginning", legend=False, figsize=(100,100), alpha=1, cmap="Spectral")
ax.set_axis_off()
ctx.add_basemap(ax, url=ctx.providers.CartoDB.DarkMatter)